In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
covid_before_df=pd.read_csv('./data/before_covid19.csv') # 10296 행
covid_df=pd.read_csv('./data/covid19.csv') # 666 행 
covid_after_df=pd.read_csv('./data/after_covid19.csv') # 529 행

In [ ]:
# Train-Test 분할을 위한 데이터 결합
train_test_df = pd.concat([covid_before_df, covid_after_df], axis=0)
train_test_df['success'] = train_test_df['ROI'].apply(lambda x: 1 if x >= 100 else 0)

In [ ]:
covid_df['ROI'] = covid_df.apply(lambda row: (row['profit'] / row['budget'])*100, axis=1)
covid_df['success'] = covid_df['ROI'].apply(lambda x: 1 if x >= 100 else 0)
covid_y_test=covid_df['success']

In [ ]:
from sklearn.model_selection import train_test_split

train_test_df.drop(['id','title','ROI','cast','director','adjusted_revenue','adjusted_budget','revenue','release_date','profit','popularity'],axis=1,inplace=True) 
covid_df.drop(['id','title','ROI','cast','director','adjusted_revenue','adjusted_budget','revenue','release_date','profit','popularity'],axis=1,inplace=True)

# Train-Validation 분할 (8:2)
X_train,X_test,y_train,y_test = train_test_split(train_test_df.drop('success',axis=1),train_test_df['success'],test_size=0.2, random_state=42, shuffle=True)

In [ ]:
def apply_one_hot_encoding(data, column_name):
    # 문자열 분리
    data[column_name] = data[column_name].str.split(', ')
    
    # 고유한 값 추출
    all_categories = set([item for sublist in data[column_name] for item in sublist])
    
    # 원핫 인코딩 적용
    for category in all_categories:
        data[category] = data[column_name].apply(lambda x: 1 if category in x else 0)
    
    # 원핫 인코딩 대상 열 제거
    data = data.drop(column_name, axis=1)
    
    return data    

In [ ]:
X_train=apply_one_hot_encoding(X_train,'genres')
X_test=apply_one_hot_encoding(X_test,'genres')
covid_df=apply_one_hot_encoding(covid_df,'genres')

In [ ]:
X_test = X_test[X_train.columns]
covid_df = covid_df[X_train.columns]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 평가 지표 계산 함수
def evaluate(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return accuracy, precision, recall, f1

# 학습

## XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

# K-Fold 설정 (5개 폴드로 교차 검증)
kf = KFold(n_splits=5, shuffle=True, random_state=2)

# XGBoost 모델 기본 설정
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=2
)

# 최적화할 하이퍼파라미터
param_dist = {
    'n_estimators': np.arange(500, 801, 100),
    'max_depth': np.arange(3, 5),
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# RandomizedSearchCV 적용 (n_iter=60, 5-fold CV, f1-score 기준)
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=60,  # 랜덤으로 60개 조합 탐색
    scoring='f1',
    cv=kf,
    verbose=1,
    n_jobs=-1,
    random_state=2
)

# 학습 진행
random_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", random_search.best_params_)

# 최적의 모델 추출
best_xgb = random_search.best_estimator_

# 학습 데이터 및 테스트 데이터 예측
y_train_pred = best_xgb.predict(X_train)
y_test_pred = best_xgb.predict(X_test)

# 학습 데이터 평가
train_accuracy, train_precision, train_recall, train_f1 = evaluate(y_train, y_train_pred)

# 테스트 데이터 평가
test_accuracy, test_precision, test_recall, test_f1 = evaluate(y_test, y_test_pred)

# 평가 결과를 데이터프레임으로 정리
evaluation_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Train': [train_accuracy, train_precision, train_recall, train_f1],
    'Test': [test_accuracy, test_precision, test_recall, test_f1]
})

# 평가 결과를 CSV 파일로 저장
evaluation_df.to_csv('model_evaluation.csv', index=False)

print(f"학습 데이터 평가:\n{evaluation_df[['Metric', 'Train']]}")
print(f"테스트 데이터 평가:\n{evaluation_df[['Metric', 'Test']]}")
print("\n평가 결과가 'model_evaluation.csv' 파일로 저장되었습니다.")


In [ ]:
# 결과를 데이터프레임으로 변환
df = pd.read_csv('model_evaluation.csv')

# 학습 데이터와 테스트 데이터의 성능 차이 계산
df['accuracy_diff'] = df['train_accuracy'] - df['test_accuracy']
df['precision_diff'] = df['train_precision'] - df['test_precision']
df['recall_diff'] = df['train_recall'] - df['test_recall']
df['f1_diff'] = df['train_f1'] - df['test_f1']

# 저장
df[['params', 'accuracy_diff', 'precision_diff', 'recall_diff', 'f1_diff']].to_csv('diff.csv',index=False)

---
파일 확인 후

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.model_selection import cross_validate, KFold

# XGBoost 모델 기본 설정
xgb_model = XGBClassifier(
    subsample=0.7, 
    n_estimators=700, 
    max_depth=3,
    learning_rate=0.01,
    colsample_bytree=0.8, 
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=2
)


# XGBoost 모델 학습
xgb_model.fit(X_train, y_train)

# K-Fold 설정 (5개 폴드로 교차 검증)
kf = KFold(n_splits=5, shuffle=True, random_state=2)

# K-Fold 교차 검증 결과 얻기
# 교차 검증 수행
scoring = ['accuracy', 'precision', 'recall', 'f1']  # 여러 지표를 평가
cv_results = cross_validate(xgb_model, X_train, y_train, cv=kf, scoring=scoring)

# 각 지표의 평균 결과 출력
print("교차 검증 결과:")
print("정확도 평균:", np.mean(cv_results['test_accuracy']))
print("정밀도 평균:", np.mean(cv_results['test_precision']))
print("재현율 평균:", np.mean(cv_results['test_recall']))
print("F1 점수 평균:", np.mean(cv_results['test_f1']))


print("훈련데이터")
# 예측 및 평가 (훈련 데이터에 대한 예측)
y_train_pred = xgb_model.predict(X_train)
evaluate(y_train,y_train_pred)


# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from sklearn.metrics import f1_score
import numpy as np

# 모델 정의
rf_model = RandomForestClassifier(
    class_weight='balanced',
    random_state=2,
    n_jobs=-1
)

# 파라미터 그리드 정의
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500,600],
    'max_depth': [3, 4, 5, 6, 7],
}

# KFold 정의 (5-fold 교차 검증)
kf = KFold(n_splits=5, shuffle=True, random_state=2)

# GridSearchCV로 파라미터 최적화
grid_search = GridSearchCV(rf_model, param_grid, cv=kf, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print("최적 파라미터:", grid_search.best_params_)


---
### 성능평가
#### XGBoost

In [ ]:
print("테스트데이터")
# 예측 및 평가 (훈련 데이터에 대한 예측)
y_test_pred = xgb_model.predict(X_test)
evaluate(y_test,y_test_pred)

In [ ]:
y_train_pred = xgb_model.predict(X_train)
evaluate(y_train,y_train_pred)

In [ ]:
# 코비드 예측
y_pred=xgb_model.predict(covid_df)
evaluate(covid_y_test,y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve


def roc_curve_plot(y_test , pred_proba_c1):
    # 임곗값에 따른 FPR, TPR 값을 반환 받음. 
    fprs , tprs , thresholds = roc_curve(y_test ,pred_proba_c1)

    # AUC 값 계산
    auc_score = roc_auc_score(y_test, pred_proba_c1)

    # ROC Curve를 plot 곡선으로 그림
    plt.figure(figsize=(7, 7))
    plt.plot(fprs, tprs, label=f'ROC Curve (AUC = {auc_score:.3f})')
    
    # 가운데 대각선 직선을 그림
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    
    # FPR X 축의 Scale을 0.1 단위로 변경, X, Y 축명 설정 등
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xlabel('FPR (1 - Specificity)')
    plt.ylabel('TPR (Recall / Sensitivity)')
    plt.legend()

    # AUC 값 텍스트로 추가
    #plt.text(0.6, 0.2, f'AUC = {auc_score:.3f}', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))

    plt.show()
    
roc_curve_plot(y_test,xgb_model.predict_proba(X_test)[:, 1] )


#### RandomForest

In [ ]:

# 최적 모델
best_rf_model = grid_search.best_estimator_

# 교차 검증 수행 (최적화된 모델로)
cv_results = cross_validate(best_rf_model, X_train, y_train, cv=kf, scoring=['accuracy', 'precision', 'recall', 'f1'])

# 각 지표의 평균 결과 출력
print("\n교차 검증 결과:")
print("정확도 평균:", np.mean(cv_results['test_accuracy']))
print("정밀도 평균:", np.mean(cv_results['test_precision']))
print("재현율 평균:", np.mean(cv_results['test_recall']))
print("F1 점수 평균:", np.mean(cv_results['test_f1']))

# train 데이터 예측 수행
print("\ntrain 데이터 예측")
y_train_pred = best_rf_model.predict(X_train)
evaluate(y_train,y_train_pred)

# test 데이터 예측 수행
print("\ntest 데이터 예측")
y_test_pred = best_rf_model.predict(X_test)
evaluate(y_test,y_test_pred)

In [ ]:
# 코비드 예측
y_pred=best_rf_model.predict(covid_df)
evaluate(covid_y_test,y_pred)

In [ ]:
roc_curve_plot(y_test,best_rf_model.predict_proba(X_test)[:, 1] )

---
### 특성중요도

In [ ]:
from xgboost import plot_importance
plot_importance(xgb_model)
plt.show()

In [ ]:
# 특성 중요도 추출
feature_importances = best_rf_model.feature_importances_

# 중요도와 특성 이름을 함께 정리
feature_names = X_train.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# 중요도가 높은 순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(feature_importance_df)

# 특성 중요도를 시각화
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title("Feature Importance", fontsize=16)
plt.show()